token

In [ ]:

from huggingface_hub import login
login(token="your--token")



for fast processing

above is successful for static video, now

for run time detection

the correct verion of this above code.

In [ ]:
import cv2
import torch
from transformers import pipeline
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow

# Configuration
CONFIG = {
    "frame_size": (640, 480),  # Reduced processing resolution
    "frame_skip": 3,  # Process every 3rd frame
    "confidence_threshold": 0.7,
    "half_precision": True,
    "batch_size": 4  # For future batch processing
}

# Initialize pipeline with optimizations
device = 0 if torch.cuda.is_available() else -1
torch_dtype = torch.float16 if (device == 0 and CONFIG["half_precision"]) else torch.float32

detector = pipeline(
    model="google/owlvit-base-patch32-v2",
    task="zero-shot-object-detection",
    device=device,
    torch_dtype=torch_dtype
)

# Optimized categories
custom_categories = [
    "lightbulb",  # Simplified descriptions
    "matchstick",
    "monitor",
    "lion",
    "gaming console"
]

def smart_resize(frame):
    """Maintain aspect ratio while resizing"""
    h, w = frame.shape[:2]
    scale = CONFIG["frame_size"][0] / w
    return cv2.resize(frame, (int(w*scale), int(h*scale)))

def process_frame(frame, prev_predictions=None):
    """
    Optimized frame processing with prediction caching
    """
    # Resize first to reduce processing load
    resized_frame = smart_resize(frame)
    orig_h, orig_w = frame.shape[:2]
    resized_h, resized_w = resized_frame.shape[:2]

    # Convert to PIL once
    pil_image = Image.fromarray(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))

    # Only process if no valid cached predictions
    if prev_predictions is None:
        with torch.no_grad():  # Disable gradient calculation
            predictions = detector(
                pil_image,
                text_prompts=custom_categories,
                threshold=CONFIG["confidence_threshold"]
            )
    else:
        predictions = prev_predictions

    # Scale boxes back to original dimensions
    scale_x = orig_w / resized_w
    scale_y = orig_h / resized_h

    annotated_frame = frame.copy()
    for pred in predictions['boxes']:
      if pred['score'] < CONFIG["confidence_threshold"]:
        continue
        box = pred
        # Scale coordinates
        xmin, ymin, xmax, ymax = map(int, box)

        # Draw bounding box
        cv2.rectangle(annotated_frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        # Label with confidence
        label = f"{pred['label']} {pred['score']:.2f}"
        cv2.putText(annotated_frame, label, (xmin, ymin-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return annotated_frame, predictions

def main(video_source="/content/drive/MyDrive/bulb_sample_video.mp4"):
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error opening video")
        return

    # Get original video properties
    orig_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    orig_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate actual output FPS
    output_fps = fps / CONFIG["frame_skip"]

    # Video writer setup
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("optimized_output.mp4", fourcc, output_fps,
                         (orig_width, orig_height))

    frame_count = 0
    prev_predictions = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if frame_count % CONFIG["frame_skip"] != 0:
            # Skip frame but reuse previous predictions
            out.write(frame)
            continue

        # Process frame
        processed_frame, prev_predictions = process_frame(frame, prev_predictions)
        out.write(processed_frame)

        # Optional display
        cv2_imshow(processed_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # from moviepy.editor import VideoFileClip
    # clip = VideoFileClip("optimized_video.mp4")
    # clip.preview()

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import torch
from transformers import pipeline
from PIL import Image

# Initialize the zero-shot object detection pipeline
device = 0 if torch.cuda.is_available() else -1
detector = pipeline(model="google/owlvit-base-patch32", task="zero-shot-object-detection", device=device)

# Define custom object categories
custom_categories = [
    "a photo of a lightbulb",
    "a photo of a matchstick",
    "a photo of a monitor",
    "a photo of a lion",
    "a photo of a gaming console"
]

def process_frame(frame):
    """
    Process a single video frame for zero-shot object detection.
    """
    # Convert OpenCV frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Perform zero-shot object detection
    predictions = detector(pil_image, candidate_labels=custom_categories)

    # Annotate the frame with bounding boxes and labels
    for pred in predictions:
        box = pred['box']
        label = pred['label']
        score = pred['score']

        # Draw bounding box
        cv2.rectangle(frame,
                      (int(box['xmin']), int(box['ymin'])),
                      (int(box['xmax']), int(box['ymax'])),
                      (0, 255, 0), 2)

        # Add label and confidence score
        text = f"{label}: {score:.2f}"
        cv2.putText(frame, text,
                    (int(box['xmin']), int(box['ymin']) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return frame

def main():
    """
    Main function to capture video from webcam and perform real-time object detection.
    """
    # Open webcam (use 0 for default camera)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        return

    print("Press 'q' to exit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from webcam.")
            break

        # Process the current frame for object detection
        annotated_frame = process_frame(frame)

        # Display the annotated frame in real-time
        cv2.imshow("Real-Time Object Detection", annotated_frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Device set to use cpu


Error: Unable to access the webcam.


another different one

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load a pre-trained model (e.g., TensorFlow Lite or custom model)
model_path = "path_to_your_model.tflite"  # Replace with your model path
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load the reference image for one-shot detection (non-COCO object)
reference_image_path = "path_to_reference_image.jpg"  # Replace with your reference image path
reference_image = cv2.imread(reference_image_path)
reference_image = cv2.resize(reference_image, (input_details[0]['shape'][1], input_details[0]['shape'][2]))
reference_image = np.expand_dims(reference_image, axis=0).astype(np.float32)

# Initialize webcam for live video feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Preprocess frame for model input
    input_frame = cv2.resize(frame, (input_details[0]['shape'][1], input_details[0]['shape'][2]))
    input_frame = np.expand_dims(input_frame, axis=0).astype(np.float32)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_frame)
    interpreter.invoke()

    # Extract predictions
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Display predictions on the live video feed
    for detection in output_data:
        confidence = detection[2]
        if confidence > 0.5:  # Threshold for confidence level
            x_min, y_min, x_max, y_max = detection[3:7]
            x_min, y_min = int(x_min * frame.shape[1]), int(y_min * frame.shape[0])
            x_max, y_max = int(x_max * frame.shape[1]), int(y_max * frame.shape[0])

            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            label = f"Object: {confidence:.2f}"
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Print predictions to console
            print(f"Detected object at [{x_min}, {y_min}, {x_max}, {y_max}] with confidence {confidence:.2f}")

    # Display the live video feed with detections
    cv2.imshow("Live Object Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


3.rd verion of code